# Import modules

In [2]:
from cam import *
import requests
import pandas as pd
import json
from json import loads

# Functions

## Get last values

In [3]:
def last_values(data_json):
    data = pd.json_normalize(data_json['patients'])
    df = data.copy()
    last_values = {}
    for col in df:
        if col == 'date':
            continue
        last_valid_idx = df[col].last_valid_index()
        last_values[col] = df.loc[last_valid_idx, col] if last_valid_idx else None

    last_values = pd.DataFrame([last_values])
    return last_values

## Request get patient information

In [4]:
def fetch_patient_information(api_url, headers=None):
    try:
        response = requests.get(api_url, headers=headers)

        if response.status_code == 200:  # Assuming 200 indicates a successful request
            patient_data = response.json()
            return patient_data
        else:
            print(f"Failed to fetch data. Status code: {response.status_code}")
            return None
    
    except requests.RequestException as e:
        print(f"Request Exception: {e}")
        return None


## Criteria comparison

In [5]:
import pandas as pd
import numpy as np

def clinical_compare_1(df1):
    # Creazione del secondo DataFrame (df2)
    num_righe = 5

    # Generazione di sequenze di valori per ogni colonna
    cbc_values = [3 + i * (8 - 3) / (num_righe - 1) for i in range(num_righe)]
    bmp_values = [75 + i * (97 - 75) / (num_righe - 1) for i in range(num_righe)]
    cmp_values = [50 + i * (80 - 50) / (num_righe - 1) for i in range(num_righe)] 


    df2 = pd.DataFrame({
        'CBC': cbc_values,
        'BMP': bmp_values,
        'CMP': cmp_values
    })

    # Mantenere solo le colonne presenti in entrambi i DataFrame
    colonne_comuni = df1.columns.intersection(df2.columns)
    df1 = df1[colonne_comuni]
    df2 = df2[colonne_comuni]

    # Controlla la presenza di valori NA
    if df1.isna().any().any() or df2.isna().any().any():
        return False

    # Confronta le colonne
    for colonna in df1.columns:
        min_val, max_val = df2[colonna].min(), df2[colonna].max()
        if not df1[colonna].between(min_val, max_val).all():
            return False

    return True


    

def clinical_compare_2(df1):
    # Creazione del secondo DataFrame (df2)
    num_righe = 5

    # Generazione di sequenze di valori per ogni colonna
    lipid_panel = [120 + i * (250- 120) / (num_righe - 1) for i in range(num_righe)]  # Valori da 4 a 7
    tfts = [1 + i * (4 - 1) / (num_righe - 1) for i in range(num_righe)]  # Valori da 80 a 85
    urinalysis = [0.5 + i * (2.5 - 0.5) / (num_righe - 1) for i in range(num_righe)]  # Valori da 60 a 80

# Creazione del DataFrame
    df2 = pd.DataFrame({
        'Lipid Panel': lipid_panel,
        'TFTs': tfts,
        'Urinalysis': urinalysis
    })

    # Mantenere solo le colonne presenti in entrambi i DataFrame
    colonne_comuni = df1.columns.intersection(df2.columns)
    df1 = df1[colonne_comuni]
    df2 = df2[colonne_comuni]

    # Controlla la presenza di valori NA
    if df1.isna().any().any() or df2.isna().any().any():
        return False

    # Confronta le colonne
    for colonna in df1.columns:
        min_val, max_val = df2[colonna].min(), df2[colonna].max()
        if not df1[colonna].between(min_val, max_val).all():
            return False

    return True

# External Task Worker

In [ ]:
import cam

class ExternalTaskWorker:
    def __init__(self):
        self.worker = cam.Client("https://digibp.herokuapp.com/engine-rest")
        # define the topic and your tenant id in the variables
        # the second parameter to the subscribe() method should match the name of the callback function
        # subscribe() calls the fetchAndLock 
        self.worker.subscribe('GetClinicalData', self.get_clinical_data, 'MI9')
        self.worker.polling()
    

    def get_clinical_data(self, taskid, response):
        try:
            user = response[0]['variables']['user']['value']
            password = response[0]['variables']['password']['value']
            url = response[0]['variables']['url']['value']
            clinical1 = response[0]['variables']['clinical1']['value']
            clinical2 = response[0]['variables']['clinical2']['value']
            print(user, password, url)
            print(response[0]['variables'])

        except:
            user = False
            password = False
            url = False

        if clinical1:
            clinical = 1

        if clinical2:
            clinical = 2

        print(clinical)

        #Initialize login
        data = {'user': user, 'password': password}

        # Sending a POST request with requests.post()
        url_login = url + '/login'
        print(url_login)
        response = requests.post(url_login, data=data)

        # Checking the response
        if response.status_code == 200:
            print("POST request was successful!")
            print("Response:", response.text)

        else:
            print("POST request failed. Status code:", response.status_code)

        resp_dict = json.loads(response.text)
        token = resp_dict['access_token']

        #Get data
        api_endpoint = url + '/patient_data'
        # If the API requires authentication, provide necessary headers (such as API key, token, etc.)
        api_headers = {'Authorization': 'Bearer '+ token}


        # Fetch patient information
        patient_info = fetch_patient_information(api_endpoint, headers=api_headers)

        if patient_info:
            # Process or display patient information
            print('basic patient info', patient_info)
            # You can iterate through patient_info and access specific fields as needed
        else:
            print("Failed to fetch patient information.")

        
        data = pd.json_normalize(patient_info['patients'])
        f_name = data['first_name'].item()
        l_name = data['last_name'].item()
        birth_date = data['date_of_birth'].item()
        patient_iddd = data['patient_id'].item()



        #Get lab data
        api_endpoint = url + '/lab_data'
        # If the API requires authentication, provide necessary headers (such as API key, token, etc.)
        api_headers = {'Authorization': 'Bearer '+ token}


        # Fetch patient information
        patient_info = fetch_patient_information(api_endpoint, headers=api_headers)

        if patient_info:
            # Process or display patient information
            print('lab data', patient_info)
            # You can iterate through patient_info and access specific fields as needed
        else:
            print("Failed to fetch patient information.")

        lab_values = last_values(patient_info)
        print(lab_values)

        if clinical == 1:
            print('Clinical 1')
            eligibility = clinical_compare_1(lab_values)
            print(eligibility)

        if clinical == 2:
            eligibility = clinical_compare_2(lab_values)
            print(eligibility)
            
        if eligibility:
            elig = "Yes"
        
        else:
            elig = "No"


        clinicaltrial = 'Clinical Trial ' + str(clinical)
        
        variables = {"patient_id": patient_iddd, 'fname': f_name, 'lname': l_name, 'date_birth': birth_date , 'clinicaltrial': clinicaltrial, 
        'eligibility': elig}
        

        
        self.worker.complete(taskid, **variables)


if __name__ == '__main__':
    ExternalTaskWorker()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=c719ecc6-7add-42e5-96f8-239e9c0d87b1' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>